In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# Checking our GPU use

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4609496314066956351, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14557074410920446435
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9430211861005811161
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15956161332
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7544216443591123209
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [5]:
!cat /proc/meminfo


MemTotal:       13335188 kB
MemFree:        10128860 kB
MemAvailable:   12197244 kB
Buffers:           75904 kB
Cached:          2109860 kB
SwapCached:            0 kB
Active:           889972 kB
Inactive:        1903868 kB
Active(anon):     539148 kB
Inactive(anon):     8520 kB
Active(file):     350824 kB
Inactive(file):  1895348 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              2692 kB
Writeback:             0 kB
AnonPages:        608104 kB
Mapped:           485128 kB
Shmem:              9092 kB
Slab:             167204 kB
SReclaimable:     126412 kB
SUnreclaim:        40792 kB
KernelStack:        4544 kB
PageTables:         7564 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6667592 kB
Committed_AS:    3466092 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

Using the Keras doc from here for IMDb sentiment analysis:

https://keras.io/examples/imdb_bidirectional_lstm/

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [0]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, CuDNNLSTM, CuDNNGRU
from keras.datasets import imdb

In [0]:
# max_features = vocabulary size
max_features = 20000
# maxlen = max length of our sentences, all are padded / cut to be size maxlen
maxlen = 100
batch_size = 32

In [22]:
print("--- Loading Data ---")
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

--- Loading Data ---
25000 train sequences
25000 test sequences


In [23]:
print('--- Pad sequences (samples x time) ---')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)

--- Pad sequences (samples x time) ---
X_train shape:  (25000, 100)
X_test shape:  (25000, 100)


In [24]:
y_train.shape

(25000,)

## Standard LSTM

In [0]:
model_lstm = Sequential()
model_lstm.add(Embedding(max_features, 128, input_length=maxlen))
model_lstm.add(CuDNNLSTM(64))
model_lstm.add(Dropout(0.2))
model_lstm.add(Dense(1, activation='sigmoid'))

In [0]:
model_lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [37]:
print('--- TRAIN ---')
model_lstm.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 10s 390us/step - loss: 0.4027 - acc: 0.8164 - val_loss: 0.3628 - val_acc: 0.8443
Epoch 2/4
25000/25000 [==============================] - 9s 362us/step - loss: 0.2338 - acc: 0.9096 - val_loss: 0.3879 - val_acc: 0.8458
Epoch 3/4
25000/25000 [==============================] - 9s 357us/step - loss: 0.1511 - acc: 0.9442 - val_loss: 0.4349 - val_acc: 0.8426
Epoch 4/4
25000/25000 [==============================] - 9s 358us/step - loss: 0.0990 - acc: 0.9646 - val_loss: 0.4909 - val_acc: 0.8352


## Bidirectional LSTM

In [0]:
model_bi_lstm = Sequential()
model_bi_lstm.add(Embedding(max_features, 128, input_length=maxlen))
model_bi_lstm.add(Bidirectional(CuDNNLSTM(64)))
model_bi_lstm.add(Dropout(0.2))
model_bi_lstm.add(Dense(1, activation='sigmoid'))

In [0]:
model_bi_lstm.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
print('--- TRAIN ---')
model_bi_lstm.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 15s 583us/step - loss: 0.4157 - acc: 0.8036 - val_loss: 0.3448 - val_acc: 0.8524
Epoch 2/4
25000/25000 [==============================] - 13s 526us/step - loss: 0.2237 - acc: 0.9125 - val_loss: 0.4193 - val_acc: 0.8353
Epoch 3/4
25000/25000 [==============================] - 13s 539us/step - loss: 0.1212 - acc: 0.9562 - val_loss: 0.4540 - val_acc: 0.8433
Epoch 4/4
25000/25000 [==============================] - 13s 536us/step - loss: 0.0658 - acc: 0.9774 - val_loss: 0.6272 - val_acc: 0.8333


## GRU

In [0]:
model_gru = Sequential()
model_gru.add(Embedding(max_features, 128, input_length=maxlen))
model_gru.add(CuDNNGRU(64))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1, activation='sigmoid'))

In [0]:
model_gru.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [46]:
print('--- TRAIN ---')
model_gru.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 10s 396us/step - loss: 0.4279 - acc: 0.7973 - val_loss: 0.3314 - val_acc: 0.8558
Epoch 2/4
25000/25000 [==============================] - 9s 346us/step - loss: 0.2230 - acc: 0.9138 - val_loss: 0.3351 - val_acc: 0.8539
Epoch 3/4
25000/25000 [==============================] - 9s 348us/step - loss: 0.1224 - acc: 0.9561 - val_loss: 0.4624 - val_acc: 0.8438
Epoch 4/4
25000/25000 [==============================] - 9s 345us/step - loss: 0.0628 - acc: 0.9786 - val_loss: 0.5055 - val_acc: 0.8395


## Bidirectional GRU

In [0]:
model_bi_gru = Sequential()
model_bi_gru.add(Embedding(max_features, 128, input_length=maxlen))
model_bi_gru.add(Bidirectional(CuDNNGRU(64)))
model_bi_gru.add(Dropout(0.2))
model_bi_gru.add(Dense(1, activation='sigmoid'))

In [0]:
model_bi_gru.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [49]:
print('--- TRAIN ---')
model_bi_gru.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 14s 554us/step - loss: 0.4133 - acc: 0.8040 - val_loss: 0.3383 - val_acc: 0.8512
Epoch 2/4
25000/25000 [==============================] - 12s 498us/step - loss: 0.2154 - acc: 0.9180 - val_loss: 0.3490 - val_acc: 0.8549
Epoch 3/4
25000/25000 [==============================] - 12s 494us/step - loss: 0.0994 - acc: 0.9650 - val_loss: 0.4801 - val_acc: 0.8456
Epoch 4/4
25000/25000 [==============================] - 12s 485us/step - loss: 0.0387 - acc: 0.9869 - val_loss: 0.6527 - val_acc: 0.8387


No significative difference between all these models

In [0]:
model_bi_gru_2 = Sequential()
model_bi_gru_2.add(Embedding(max_features, 128, input_length=maxlen))
model_bi_gru_2.add(Bidirectional(CuDNNGRU(64)))
model_bi_gru_2.add(Dropout(0.5))
model_bi_gru_2.add(Dense(50, activation='sigmoid'))
model_bi_gru_2.add(Dropout(0.5))
model_bi_gru_2.add(Dense(1, activation='sigmoid'))

In [0]:
model_bi_gru_2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [53]:
print('--- TRAIN ---')
model_bi_gru_2.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 14s 578us/step - loss: 0.4440 - acc: 0.7867 - val_loss: 0.3547 - val_acc: 0.8456
Epoch 2/4
25000/25000 [==============================] - 13s 503us/step - loss: 0.2393 - acc: 0.9098 - val_loss: 0.3435 - val_acc: 0.8547
Epoch 3/4
25000/25000 [==============================] - 13s 504us/step - loss: 0.1361 - acc: 0.9565 - val_loss: 0.4264 - val_acc: 0.8445
Epoch 4/4
25000/25000 [==============================] - 13s 502us/step - loss: 0.0721 - acc: 0.9793 - val_loss: 0.5152 - val_acc: 0.8358


We can see abit of overfitting, let's try to increase the Dropout rate to 0.5

In [57]:
model_bi_gru_3 = Sequential()
model_bi_gru_3.add(Embedding(max_features, 128, input_length=maxlen))
model_bi_gru_3.add(Bidirectional(CuDNNGRU(64)))
model_bi_gru_3.add(Dropout(0.9))
model_bi_gru_3.add(Dense(50, activation='sigmoid'))
model_bi_gru_3.add(Dropout(0.9))
model_bi_gru_3.add(Dense(1, activation='sigmoid'))

In [0]:
model_bi_gru_3.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [59]:
print('--- TRAIN ---')
model_bi_gru_3.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[X_test, y_test])

--- TRAIN ---
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 15s 595us/step - loss: 0.7742 - acc: 0.4972 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/4
25000/25000 [==============================] - 13s 508us/step - loss: 0.6965 - acc: 0.5022 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 3/4
25000/25000 [==============================] - 13s 502us/step - loss: 0.6944 - acc: 0.5020 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/4
25000/25000 [==============================] - 13s 502us/step - loss: 0.6938 - acc: 0.4992 - val_loss: 0.6931 - val_acc: 0.5000


In [0]:
df_pred.to_csv('submission.csv', index=False, header=True)

In [0]:
import os
print( os.getcwd() )
print( os.listdir() )

In [0]:
from google.colab import files
files.download( "submission.csv" )  